In [8]:
# Import packages
import requests
from bs4 import BeautifulSoup
import re
from random import randint
from time import sleep

In [9]:
# Url of the website to scrape
URL = "https://www.yelp.com/search?find_desc=sushi&find_loc=San+Francisco"

In [10]:
# GET is used to request data from a specified resource.
page = requests.get(URL)

In [4]:
# To see the content in string format, use .text
print(page.text)

<!DOCTYPE html><html lang="en-US" prefix="og: http://ogp.me/ns#" style="margin: 0;padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline;"><head><script>document.documentElement.className=document.documentElement.className.replace(/no-js/,"js");</script><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><meta http-equiv="Content-Language" content="en-US" /><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"><link rel="mask-icon" sizes="any" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/b2bb2fb0ec9c/assets/img/logos/yelp_burst.svg" content="#FF1A1A"><link rel="shortcut icon" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/b05852393ae5/assets/img/logos/favicon.ico"><script> window.ga=window.ga||function(){(ga.q=ga.q||[]).push(arguments)};ga.l=+new Date;window.ygaPageStartTime=new Date().getTime();</script><script async src="https://www.google-analytics.com/analytics.

In [11]:
# To see the response’s content in bytes, you use .content:
soup = BeautifulSoup(page.content, "html.parser")

In [12]:
# Find the total page number
page_element = soup.find("div", class_ = "border-color--default__09f24__NPAKY text-align--center__09f24__fYBGO")
page_num = page_element.find("span", class_ = "css-chan6m").text
page_num

'1 of 24'

In [13]:
# Use regex to find the last number in the above string
page_format = "(of )(\d+)"
total_page_num = int(re.search(page_format, page_num).group(2))
total_page_num

24

In [14]:
# Create an url for each page
urls = []
pages = list(range(0, total_page_num + 1 ))
for page in pages:
    url = 'https://www.yelp.com/search?find_desc=sushi&find_loc=San+Francisco%2C+CA&start=' + str(page*10)
    urls.append(url)

In [15]:
# Create an function for scraping the front page of yelp
def webscrapefrontpage(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    
    # Scrape restaurant names
    names = soup.find_all("span", class_="css-1egxyvc")
    for name in names:
        a_tag = name.find("a", class_ = "css-1m051bw")
        if a_tag:
            restaurant_name =a_tag.getText()
            name_list.append(restaurant_name)
            
    # Scrape review counts
    review_counts = soup.find_all("span", class_="reviewCount__09f24__tnBk4 css-chan6m")
    for review_count in review_counts:
        review = int(review_count.text)
        review_count_list.append(review)

    # Scrape locations
    p_tags = soup.find_all("p", class_="css-dzq7l1")
    for p_tag in p_tags:
        location = p_tag.find("span", class_="css-chan6m" )
        if location:
            location_label = location.text
        else:
            location_label = None
    
        price = p_tag.find("span", class_="priceRange__09f24__mmOuH css-1s7bx9e" )
        if price:
            price_label = price.text
        else:
            price_label = None
        price_list.append(price_label)
        location_list.append(location_label)
 
    # Scrape ratings
    rating_format = "(^\d\.\d)|(\d)"
    ratings = soup.select('div[class*="i-stars-"]')
    for rating in ratings:
        rating_label = rating["aria-label"]
        rating_number = re.search(rating_format, rating_label).group(0)
        rating_list.append(rating_number)
    
    # Scrape price range
    prices = soup.find_all("span", class_ = "priceRange__09f24__mmOuH css-1s7bx9e")
    for price in prices:
        if price:
            price_label = price.text
            price_list.append(price_label)
        else: price_list.append("N/A")
    
    sleep(randint(1,10))    

In [16]:
name_list = []
review_count_list = []
location_list = []
rating_list = []
price_list = []

for url in urls:
    webscrapefrontpage(url)


In [17]:
# Scrape restaurant names
names = soup.find_all("span", class_="css-1egxyvc")
name_list = []
for name in names:
    a_tag = name.find("a", class_ = "css-1m051bw")
    if a_tag:
        restaurant_name =a_tag.getText()
        name_list.append(restaurant_name)
        print(restaurant_name)

Nara Restaurant & Sake Bar
Tataki
Toyama Sushi
Sakesan Sushi & Bistro
Kazan
Kibatsu
Ebisu
Dash Japanese Tapas & Sushi
Little Sushi Bar
Mamanoko


In [19]:
review_counts = soup.find_all("span", class_="reviewCount__09f24__tnBk4 css-chan6m")
print(review_counts)

[<span class="reviewCount__09f24__tnBk4 css-chan6m">1765</span>, <span class="reviewCount__09f24__tnBk4 css-chan6m">1342</span>, <span class="reviewCount__09f24__tnBk4 css-chan6m">339</span>, <span class="reviewCount__09f24__tnBk4 css-chan6m">1138</span>, <span class="reviewCount__09f24__tnBk4 css-chan6m">523</span>, <span class="reviewCount__09f24__tnBk4 css-chan6m">264</span>, <span class="reviewCount__09f24__tnBk4 css-chan6m">1915</span>, <span class="reviewCount__09f24__tnBk4 css-chan6m">17</span>, <span class="reviewCount__09f24__tnBk4 css-chan6m">119</span>, <span class="reviewCount__09f24__tnBk4 css-chan6m">395</span>]


In [20]:
# Scrape review counts
review_count_list = []
for review_count in review_counts:
    review = int(review_count.text)
    review_count_list.append(review)
    print(review)

1765
1342
339
1138
523
264
1915
17
119
395


In [21]:
# Scrape locations and prices
location_list = []
p_tags = soup.find_all("p", class_="css-dzq7l1")
for p_tag in p_tags:
    location = p_tag.find("span", class_="css-chan6m" )
    if location:
        locction_label = location.text
    else:
        locction_label = None
    
    price = p_tag.find("span", class_="priceRange__09f24__mmOuH css-1s7bx9e" )
    if price:
        price_label = price.text
    else:
        price_label = None
    print(location_label)
    print(price_label)
    


Lower Haight
Lower Pacific Heights
Inner Richmond
Inner Richmond
Mission
Hayes Valley
Inner Sunset
Noe Valley
Outer Richmond
Marina/Cow Hollow


In [22]:
rating_format = "(^\d\.\d)|(\d)"
rating_list = []
# Scrape ratings
ratings = soup.select('div[class*="i-stars-"]')
for rating in ratings:
    rating_label = rating["aria-label"]
    rating_number = re.search(rating_format, rating_label).group(0)
    rating_list.append(rating_number)
    print(rating_number)

4.5
4
4.5
4
4.5
4.5
4
5
4.5
4.5


In [67]:
print(rating_label)

4.5 star rating


In [62]:
# Scrape price range
prices = soup.find_all("span", class_ = "priceRange__09f24__mmOuH css-1s7bx9e")
for price in prices:
    print(price.text)


$$
$$
$$
$$$
$$
$$
$$
$$
$$


In [15]:
# pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [14]:
location_list

['Lower Haight',
 'Inner Richmond',
 'Noe Valley',
 'Marina/Cow Hollow',
 'Lower Pacific Heights',
 'Inner Sunset',
 'Hayes Valley',
 'Castro',
 'Noe Valley',
 'Lower Pacific Heights',
 'Mission Bay',
 'Inner Richmond',
 'Excelsior',
 'Japantown',
 'Stonestown',
 'Mission Bay',
 'Lower Nob Hill',
 'Mission',
 'Lower Haight',
 'Financial District',
 'Castro',
 'Outer Sunset',
 'The Haight',
 'Mission',
 '$$',
 'Inner Sunset',
 'Lower Nob Hill',
 'Lower Pacific Heights',
 'SoMa',
 'Marina/Cow Hollow',
 'Nob Hill',
 'Mission Bay',
 'Mission',
 'Financial District',
 'SoMa',
 'Mission',
 'Tenderloin',
 'Lower Nob Hill',
 'Lower Nob Hill',
 'NoPa',
 'Potrero Hill',
 'Lower Nob Hill',
 'Mission',
 'Financial District',
 'Ingleside Terraces',
 'Mission Bay',
 'Castro',
 'Outer Richmond',
 'Castro',
 'Outer Richmond',
 'Inner Richmond',
 'Russian Hill',
 '$$',
 'Inner Richmond',
 'Inner Richmond',
 'Mission',
 'Castro',
 'Mission',
 'Bernal Heights',
 'Westwood Park',
 'Laurel Heights',
 'Mari

In [25]:
# Replace wrong entries with empty string
location_list = list(map(lambda st: str.replace(st,"$", ""), location_list))

In [21]:
# Import scraped data into MySQL 
import mysql.connector

cnx = mysql.connector.connect(user='root', password='your password',
                              host='127.0.0.1',
                              database='sushi')
cursor = cnx.cursor()

TABLES = {}
TABLES['restaurant'] = (
    "CREATE TABLE `restaurant` ("
    "  `restaurant_id` int NOT NULL AUTO_INCREMENT,"
    "  `name` varchar(100) NOT NULL,"
    "  `review_count` int NOT NULL,"
    "  `location` varchar(100),"
    "  `price` varchar(100),"
    "  `rating` float(2,1) NOT NULL,"
    "  PRIMARY KEY (`restaurant_id`)"
    ") ENGINE=InnoDB")
cursor.execute(TABLES['restaurant'])

In [22]:
insert_records = []
for i in range(len(name_list)):
    insert_records.append([name_list[i],review_count_list[i],location_list[i],price_list[i],rating_list[i]])
print(insert_records)
    

[['Nara Restaurant & Sake Bar', 1766, 'Lower Haight', '$$', '4.5'], ['Toyama Sushi', 339, 'Inner Richmond', '$$', '4.5'], ['Saru Sushi Bar - Noe Valley', 915, 'Noe Valley', '$$$', '4.5'], ['Mamanoko', 395, 'Marina/Cow Hollow', '$$', '4.5'], ['Tataki', 1342, 'Lower Pacific Heights', '$$', '4'], ['Ebisu', 1916, 'Inner Sunset', '$$', '4'], ['Kibatsu', 264, 'Hayes Valley', '$$$', '4.5'], ['Eiji', 1118, 'Castro', '$$', '4'], ['Dash Japanese Tapas & Sushi', 17, 'Noe Valley', None, '5'], ['Akira Japanese Restaurant', 457, 'Lower Pacific Heights', '$$', '4'], ['Hakashi Japanese Sushi Bar & Grill', 397, 'Mission Bay', '$$', '4.5'], ['Sakesan Sushi & Bistro', 1138, 'Inner Richmond', '$$', '4'], ['Amami Sushi Bistro', 200, 'Excelsior', '$$$', '4.5'], ['Kui Shin Bo', 854, 'Japantown', '$$', '4'], ['Kura Revolving Sushi Bar', 188, 'Stonestown', '$$', '3.5'], ['Wabi-Sabi SF', 62, 'Mission Bay', '$$', '5'], ['Ryoko’s', 3173, 'Lower Nob Hill', '$$$', '4'], ['Kazan', 523, 'Mission', '$$', '4.5'], ['Ken

In [23]:
add_restaurant = ("INSERT INTO restaurant "
               "(name, review_count, location, price, rating) "
               "VALUES ( %s, %s, %s, %s,%s)")
cursor.executemany(add_restaurant, insert_records)
cnx.commit()
cursor.close()
cnx.close()

In [2]:
from selenium import webdriver
DRIVER_PATH = 'F:/softwares downloaded/chromedriver'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
driver.get('https://www.yelp.com/search?find_desc=sushi&find_loc=San+Francisco%2C+CA&start=0')

<ipython-input-2-0e6bc5484c5c>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=DRIVER_PATH)


In [17]:
driver.find_element_by_link_text(name_list[0]).click()

<ipython-input-17-56690ec17256>:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_link_text(name_list[0]).click()
